### Setup

In [ ]:
#download the image data
%cd /content/
!gdown https://drive.google.com/uc?id=1HpFpsQ7VojeJbXj8i9i8IeIzXrzQ5zOs
!unzip "nerf_data.zip"; rm "nerf_data.zip"
# !gdown https://drive.google.com/uc?id=1ahp0Ac3I_7Fm1usqPT_Rvgww5KaLvo6Y
# !unzip davis.zip; rm davis.zip

In [ ]:
%cd /content
!git clone https://github.com/rohaldb/Neural-Scene-Flow-Fields
%cd /content/Neural-Scene-Flow-Fields
!pip install -r requirements.txt

In [3]:
%cd /content
!git clone https://github.com/rohaldb/multi-mono-sf.git
%cd multi-mono-sf
!chmod -R +x ./
!pip install -r requirements.txt

     |████████████████████████████████| 61kB 3.2MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Execution
First run the standard preprocessing pipeline of the neural-scene-flow paper. This will:
1. Convert the intrinsics/extrinsics to a special readable format
2. Resize the images and run a single view depth model to produce disparity estimates
3. Produce optical flow estimates
4. Use the optical flow estimates to obtain motion masks

We do not use the depth map or the optical flow for custom experiments. 

In [ ]:
%cd /content/Neural-Scene-Flow-Fields/nsff_scripts
# create camera intrinsics/extrinsic format for NSFF, same as original NeRF where it uses imgs2poses.py script from the LLFF code: https://github.com/Fyusion/LLFF/blob/master/imgs2poses.py
!python save_poses_nerf.py --data_path "/content/nerf_data/kid-running/dense/"
# Resize input images and run single view model
!python run_midas.py --data_path "/content/nerf_data/kid-running/dense/" --input_w 640 --input_h 360 --resize_height 288
# Run optical flow model (for easy setup and Pytorch version consistency, we use RAFT as backbond optical flow model, but should be easy to change to other models such as PWC-Net or FlowNet2.0)
!./download_models.sh
!python run_flows_video.py --model models/raft-things.pth --data_path /content/nerf_data/kid-running/dense/ --epi_threshold 1.0 --input_flow_w 768 --input_semantic_w 1024 --input_semantic_h 576

Next we run the mono-sf code, which will produce:
1. Disparity maps
2. Scene flow estimates
3. Induced optical flow estimates

By default, it will not produce visualisations to acompany the stored tensors. To include them in the output, add `--save_vis=True` to `eval_kid_running.sh`

In [8]:
# Run mono-sf to obtain scene flow, induced optical flow and disparity estimates
%cd /content/multi-mono-sf/scripts
!./eval_kid_running.sh

/content/multi-mono-sf/scripts
2021-07-06 01:24:04 ==> Commandline Arguments
2021-07-06 01:24:04   batch_size: 1
2021-07-06 01:24:04   batch_size_val: 1
2021-07-06 01:24:04   calculate_disparity_scale: False
2021-07-06 01:24:04   checkpoint: checkpoints/checkpoint_kitti_selfsup.ckpt
2021-07-06 01:24:04   checkpoint_exclude_params: ['']
2021-07-06 01:24:04   checkpoint_include_params: ['*']
2021-07-06 01:24:04   checkpoint_mode: resume_from_latest
2021-07-06 01:24:04   conv_padding_mode: zeros
2021-07-06 01:24:04   correlation_cuda_enabled: False
2021-07-06 01:24:04   cuda: True
2021-07-06 01:24:04   evaluation: True
2021-07-06 01:24:04   finetuning: False
2021-07-06 01:24:04   lr_scheduler: None
2021-07-06 01:24:04   model: MonoSceneFlow_Multi
2021-07-06 01:24:04   num_workers: 16
2021-07-06 01:24:04   optimizer: Adam
2021-07-06 01:24:04   optimizer_amsgrad: False
2021-07-06 01:24:04   optimizer_betas: (0.9, 0.999)
2021-07-06 01:24:04   optimizer_eps: 1e-08
2021-07-06 01:24:04   optimi

Next, remove the unneeded content (nsf scene flow & nsf disparity), zip the content and store in google drive.

In [ ]:
# Remove irrelevant folders
!rm -rf /content/nerf_data/kid-running/dense/flow_i1 #nsf optical flow
!rm -rf /content/nerf_data/kid-running/dense/disp #nsf disparity

In [10]:
#save to gdrive
cd /content/
!zip -r custom_nerf_data.zip /content/nerf_data
!cp -r custom_nerf_data.zip /content/gdrive/MyDrive/Thesis/Data
!ls /content/gdrive/MyDrive/Thesis/Data